# MQTT Demo — ESP32 Device Control via Python

This Jupyter notebook demonstrates how to control **ESP32-POE-ISO nodes** (pump, ultrasonic, heater, pH probe, bio-reactor) using the Python MQTT API provided in `iot_mqtt.py`.

It includes examples for:

- Starting an MQTT broker (Mosquitto)
- Running the controller heartbeat beacon
- Connecting to device nodes
- Issuing commands (pump/heater/ultra/pH/bio)
- Reading sensor data
- Running automated experiments
- Collecting & plotting pH time-series

> **Recommended:** Run this notebook while all ESP32 nodes are powered.
> The notebook assumes that devices already have working firmware and MQTT credentials.

---

## 1) Installation

### Python Dependencies


In [1]:
!python -m pip install paho-mqtt


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


---

## 2) Start Broker and Controllers

In [ ]:
import time
from iot_mqtt import (
    PumpMQTT, UltraMQTT, HeatMQTT, PhMQTT, BioMQTT,
    start_broker_if_needed, stop_broker,
    ControllerBeacon, _best_effort_all_off
)

# ─────────────────────────────────────────────
# 1) Start broker if needed
# ─────────────────────────────────────────────
proc = start_broker_if_needed()   # No-op if already running

broker = "192.168.0.100"

# ─────────────────────────────────────────────
# 2) Controller beacon (heartbeat + ONLINE/OFFLINE)
# ─────────────────────────────────────────────
beacon = ControllerBeacon(
    broker=broker, port=1883,
    username="pyctl-controller", password="controller",
    client_id="pyctl-controller",
    status_topic="pyctl/status",
    heartbeat_topic="pyctl/heartbeat",
    heartbeat_interval=5.0,
    keepalive=30,
)
beacon.start()

# ─────────────────────────────────────────────
# 3) Device MQTT Clients
# ─────────────────────────────────────────────
pumps = PumpMQTT(
    broker=broker, username="pyctl-controller", password="controller",
    base_topic="pumps/01", client_id="pyctl-pumps"
)
ultra = UltraMQTT(
    broker=broker, username="pyctl-controller", password="controller",
    base_topic="ultra/01", client_id="pyctl-ultra"
)
heat = HeatMQTT(
    broker=broker, username="pyctl-controller", password="controller",
    base_topic="heat/01", client_id="pyctl-heat"
)
ph = PhMQTT(
    broker=broker, username="pyctl-controller", password="controller",
    base_topic="ph/01", client_id="pyctl-ph"
)
bio = BioMQTT(
    broker=broker, username="pyctl-controller", password="controller",
    base_topic="bio/01", client_id="pyctl-bio"
)

# ─────────────────────────────────────────────
# 4) Connect all devices
# ─────────────────────────────────────────────
pumps.start()
ultra.start()
heat.start()
ph.start()
bio.start()

time.sleep(1)  # allow MQTT connections to stabilize
print("Connected to broker + all device clients.")

[broker] Logging to: C:\Users\13538\OneDrive\Desktop\Arduino\mosq-python.log
[broker] Starting Mosquitto (-v for logs)...
[broker] Broker is ready.


TimeoutError: timed out

---

## 3) Check Node Status


In [ ]:
# Show live status from each node for a couple seconds
pumps.status(seconds=2.0)
time.sleep(1)
ultra.status(seconds=2.0)
time.sleep(1)
heat.status(seconds=2.0)
time.sleep(1)
ph.status(seconds=2.0)

This displays:

* heartbeat
* online/offline
* recent state messages

---

## 4) Device Examples

### Pump control

In [3]:
# Pumps
pumps.on(3)                 # pump 3 ON
time.sleep(1)
pumps.off(3)
time.sleep(1)

pumps.on(2)                 # pump 2 ON
time.sleep(1)
pumps.off(2)
time.sleep(1)

pumps.on(1, 2000)           # pump 1 ON for 2 s (auto-off)

[pyctl-pumps] Published 'ON' to pumps/01/cmd/3
[pyctl-pumps] Published 'OFF' to pumps/01/cmd/3
[pyctl-pumps] Published 'ON' to pumps/01/cmd/2
[pyctl-pumps] Published 'OFF' to pumps/01/cmd/2
[pyctl-pumps] Published 'ON:2000' to pumps/01/cmd/1


### Ultrasonic control

In [ ]:
# Ultrasonic
ultra.on(1)       # ultrasonic ch1 ON for 30 s
time.sleep(30)
ultra.off(1)      # ultrasonic ch1 OFF
time.sleep(1)

ultra.on(2)       # ultrasonic ch2 ON for 30 s
time.sleep(30)
ultra.off(2)      # ultrasonic ch2 OFF
time.sleep(1)

ultra.on(1, 15000)  # ultrasonic ch1 ON for 15 s (auto-off)

[pyctl-ultra] Published 'ON' to ultra/01/cmd/1


### Heater Control + Temperature Read

In [ ]:
# Heater + thermistor demo
heat.set_target(1, 42.0)         # set target to 42C (ESP retains on set/1)
heat.pid_on(1)                   # enable PID loop on ESP
# actively request a reading now:
try:
    for i in range(50):
        t = heat.get_base_temp(1, timeout_s=5.0)
        print("Temp(ch1) =", t)
        time.sleep(1)
except TimeoutError as e:
    print("Temp read timeout:", e)

heat.pid_off(1)                  # stop PID
heat.set_pwm(1, 0)               # ensure PWM is off
heat.off(1)                      # ensure relay is off

[pyctl-heat] Published 'SET:42.0' to heat/01/cmd/1
[pyctl-heat] Published 'PID:ON' to heat/01/cmd/1
[pyctl-heat] Published 'PID:OFF' to heat/01/cmd/1
[pyctl-heat] Published 'PWM:0' to heat/01/cmd/1
[pyctl-heat] Published 'OFF' to heat/01/cmd/1


### pH Probe

In [ ]:
# pH demo
ph.oneshot(seconds=3.0) # Single read 

# Polling @5s for ~20s and collects output
series = ph.watch_poll(interval_ms=5000, seconds=20.0, collect=True)
print("Collected points:", len(series))
for ts, val in series:
    print(ts, val)

Quick plot: 

In [ ]:
import matplotlib.pyplot as plt

ts = [x[0] for x in series]
vals = [float(x[1]) for x in series]

plt.plot(ts, vals, marker='o')
plt.xlabel("Time (s)")
plt.ylabel("pH")
plt.grid(True)
plt.title("pH over time")
plt.show()

### Biologic control

In [ ]:
# Biologic demo
bio.on(1)        # turn on channel 1
time.sleep(10)   # wait for the experiment  
bio.off(1)       # turn off channel 1

### Run everything simultaneously

In [ ]:
set_time = 60000    # ms
pumps.on(1, set_time)           # pump 1 ON for 2 s (auto-off)
pumps.on(2, set_time)           # pump 1 ON for 2 s (auto-off)
pumps.on(3, set_time)           # pump 1 ON for 2 s (auto-off)

ultra.on(2, set_time)           # ultrasonic ch2 ON for 30 s

heat.set_target(1, 42.0)        # set target to 42C (ESP retains on set/1)
heat.pid_on(1)                  # enable PID loop on ESP
time.sleep(set_time/1000)
heat.pid_off(1)                 # stop PID
heat.set_pwm(1, 0)              # ensure PWM is off
heat.off(1)                     # ensure relay is off

[pyctl-pumps] Published 'ON:60000' to pumps/01/cmd/1
[pyctl-pumps] Published 'ON:60000' to pumps/01/cmd/2
[pyctl-pumps] Published 'ON:60000' to pumps/01/cmd/3


AttributeError: 'UltraMQTT' object has no attribute 'on_for'

## 5) Disconnect + Cleanup


In [ ]:
# Turns off everything
_best_effort_all_off(pumps, ultra, heat, ph)

# Disconnect all controllers
beacon.stop()

pumps.disconnect()
ultra.disconnect()
heat.disconnect()
ph.disconnect()

# Stop broker
stop_broker(proc)

[ctl] Stopping controller beacon (OFFLINE)...
[broker] Stopping Mosquitto...
[2025-10-23 11:55:44] [broker] Mosquitto stopped.


# Notes

✅ **Controller Beacon must run**
ESP32 nodes will disable themselves if beacon heartbeats stop.

✅ **Auto-OFF**
Relays have a lease time; using `on(ch, ms)` is safer.

✅ **pH raw commands**

```python
ph.cmd_raw("Cal,mid,7.00")
ph.cmd_raw("i")          # device info
ph.cmd_raw("Status,?")   # diagnostics
```

✅ Collected pH data can be plotted or logged automatically.

---

# Troubleshooting

| Issue                | Fix                            |
| -------------------- | ------------------------------ |
| No MQTT connection   | Check broker IP + creds        |
| No pH readings       | Ensure UART wiring + Atlas EZO |
| Ultrasonic kills I²C | Add shielding / grounding      |
| Temp read timeout    | Check thermistor wiring        |
| STOP not working     | Make sure START was used       |

---

# License

MIT © 2025 Alan Yang